# LLoCa Quickstart
# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heidelberg-hepml/lloca/blob/main/examples/demo_transformer.ipynb)

In this tutorial, we give a quick introduction for how to use Lorentz Local Canonicalization (LLoCa).

LLoCa is a framework to make any network Lorentz-equivariant. It uses the concept of canonicalization, i.e. local frames where features are invariant under symmetry transformations, making it possible to process them with any backbone architecture without violating equivariance. The frames are constructed from a set of vectors constructed with a simple Lorentz-equivariant network, which are turned into Lorentz transformations through a orthonormalization step. To maximise expressivity, each particle gets its own *local* frame, but this requires a modification of message-passing to allow the communication of tensorial messages between particles in different frames.

We will now demonstrate how to build a simple LLoCa-Transformer following three steps:

1. Construct local frames based on 3 equivariantly predicted vectors
2. Transform particle features into local frames
3. Process local particle features with any backbone architecture

![LLoCa workflow](https://raw.githubusercontent.com/heidelberg-hepml/lloca/main/img/lloca.png)

In [ ]:
# install the lloca package
%pip install lloca

### 0) Generate particle data

We start by generating toy particle data, for instance for an amplitude regression task. We describe particles by a four-momentum and one scalar feature, for instance the particle type. Using random numbers, we generate a batch of 128 events with 10 particles each.

In [2]:
# generate particle data
import torch
num_scalars = 1
B, N = 128, 10
mass = 1
p3 = torch.randn(B, N, 3)
fourmomenta = torch.cat([(mass**2 + (p3**2).sum(dim=-1, keepdims=True)).sqrt(), p3], dim=-1)
scalars = torch.randn(B, N, num_scalars)
print(fourmomenta.shape, scalars.shape)

torch.Size([128, 10, 4]) torch.Size([128, 10, 1])


### 1) Construct local frames based on 3 equivariantly predicted vectors

Based on these particle features, we want to construct a local frame $L$ for each particle. The local frames are Lorentz transformations, i.e. they satisfy $L^TgL=g$ with $L\in \mathbb{R}^{4\times 4}$. We further design them to satisfy the transformation behavior $L\overset{\Lambda}{\to} L\Lambda^{-1}$ under Lorentz transformations $\Lambda$, this ensures that particle features in the local frame are invariant.

We construct the local frames in two steps. First, we use a simple Lorentz-equivariant network, `equivectors`, to construct 3 vectors.

In [3]:
from lloca.equivectors.equimlp import EquiMLP

def equivectors_constructor(n_vectors):
    return EquiMLP(
        n_vectors=n_vectors,
        num_blocks=2,
        num_scalars=num_scalars,
        hidden_channels=8,
        num_layers_mlp=2,
    )

# quickly test it
equivectors_test = equivectors_constructor(3)
vectors = equivectors_test(fourmomenta, scalars)
print(vectors.shape)

/home/jspinner/.local/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/jspinner/.local/lib/python3.10/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/jspinner/.local/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/jspinner/.local/lib/python3.10/site-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/home/jspinner/.local/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/jspinner/.local/lib/python3.10/site-p

torch.Size([128, 10, 3, 4])


This `equivectors` network is used in the larger `framesnet` class, which also includes the subsequent orthonormalization to construct the local `frames`. We therefore construct the `equivectors_constructor` to the `LearnedPDFrames` `framesnet`, which orthonormalizes the three vectors to construct a Lorentz transformation.

In [4]:
from lloca.framesnet.equi_frames import LearnedPDFrames

framesnet = LearnedPDFrames(equivectors=equivectors_constructor)
frames = framesnet(fourmomenta, scalars)
print(frames.shape)

torch.Size([128, 10, 4, 4])


Lets check that the `frames` object satisfies the Lorentz condition $L^T gL=g$

In [5]:
from lloca.utils.lorentz import lorentz_metric

metric = lorentz_metric(frames.shape[:-2])
print(metric[0,0])

lhs = torch.einsum("...ij,...jk,...kl->...il", frames.matrices, metric, frames.matrices.transpose(-1, -2))
print(lhs[0,0])

tensor([[ 1.,  0.,  0.,  0.],
        [ 0., -1., -0., -0.],
        [ 0., -0., -1., -0.],
        [ 0., -0., -0., -1.]])
tensor([[ 1.0000e+00, -5.3085e-08,  1.3039e-08,  3.6322e-08],
        [-5.3085e-08, -1.0000e+00, -2.9802e-08, -2.9802e-08],
        [ 1.3039e-08, -2.9802e-08, -1.0000e+00,  0.0000e+00],
        [ 3.6322e-08, -2.9802e-08,  0.0000e+00, -1.0000e+00]],
       grad_fn=<SelectBackward0>)


The package implements many alternative `framesnet` choices. First, a subset of the vectors can be fixed to construct frames equivariant under subgroups of the Lorentz group such as `LearnedSO3Frames` and `LearnedSO2Frames`. Second, random global frames can be sampled to implement data augmentation as in `RandomFrames`, and the baseline `IdentityFrames`.

### 2) Transform particle features into local frames

Once the frames are constructed, we have to transform the particle features into their local frames. We use the local frames transformation for the four-momenta, whereas the scalar features are already invariant by definition.

In [6]:
from lloca.reps.tensorreps_transform import TensorReps, TensorRepsTransform

fourmomenta_rep = TensorReps("1x1n")
trafo_fourmomenta = TensorRepsTransform(fourmomenta_rep)
fourmomenta_local = trafo_fourmomenta(fourmomenta, frames)
print(fourmomenta_local.shape)

features_local = torch.cat([fourmomenta_local, scalars], dim=-1)
print(features_local.shape)

torch.Size([128, 10, 4])
torch.Size([128, 10, 5])


The `lloca` package implements arbitrary Lorentz tensors through the `TensorReps` class, and their transformation behavior with `TensorRepsTransform`. We denote `0n` for scalar, `1n` for vector, `2n` for rank 2 tensor, and so on, where the `n` stands for *normal* in contrast to *parity-odd* (not fully supported). General representations can be obtained by linear combinations of these fundamentals, e.g.

In [7]:
for reps in ["1x0n", "1x1n", "1x2n", "4x0n+8x1n+3x2n+2x3n"]:
    print(f"{reps}: {TensorReps(reps).dim}-dimensional")

1x0n: 1-dimensional
1x1n: 4-dimensional
1x2n: 16-dimensional
4x0n+8x1n+3x2n+2x3n: 212-dimensional


As a cross-check, we apply a global Lorentz transformation `random` onto the fourmomenta to obtain `fourmomenta_prime`. We then re-evaluate the frames as `frames_prime`, and obtain `fourmomenta_prime_local` after transforming into the local frames. We indeed find that the four-momenta in the local frame are invariant under (global) Lorentz transformations of the original four-momenta

In [8]:
from lloca.utils.rand_transforms import rand_lorentz
from lloca.framesnet.frames import Frames

random = Frames(rand_lorentz([B,1])).repeat(1, N, 1, 1)
fourmomenta_prime = trafo_fourmomenta(fourmomenta, random)
frames_prime = framesnet(fourmomenta_prime, scalars)
fourmomenta_prime_local = trafo_fourmomenta(fourmomenta_prime, frames_prime)
print(fourmomenta_local[0,0])
print(fourmomenta_prime_local[0,0])

tensor([ 1.3665,  0.3767,  0.4872, -0.6987], grad_fn=<SelectBackward0>)
tensor([ 1.3665,  0.3767,  0.4868, -0.6989], grad_fn=<SelectBackward0>)


### 3) Process local particle features with any backbone architecture

Given the particle features in the local frame, we can process them with any backbone architecture without violating Lorentz-equivariance. To obtain an equivariant prediction, we have to finally transform the output features from the local into the global frames, however this step is trivial if the output features are scalar.

There is one caveat regarding the backbone architecture: To allow a meaningful message-passing, we have to properly transform particle features when they are communicated between particles. This manifests in a modification of the attention mechanism for transformers, and in the message-passing for graph networks. This aspect is already implemented in the backbones available in `lloca/backbone/`, and has to be added for new backbone architectures within LLoCa.

In [9]:
from lloca.backbone.transformer import Transformer

backbone = Transformer(
    in_channels=4+num_scalars,
    attn_reps="4x0n+1x1n",
    out_channels=1,
    num_blocks=2,
    num_heads=2,
)

out = backbone(features_local, frames)
print(out.shape)

torch.Size([128, 10, 1])


Finally, we check that the network output is indeed invariant under Lorentz transformations.

In [10]:
print(out[0,0])

features_prime_local = torch.cat([fourmomenta_prime_local, scalars], dim=-1)
out_prime = backbone(features_prime_local, frames_prime)
print(out_prime[0,0])

tensor([0.3903], grad_fn=<SelectBackward0>)
tensor([0.3902], grad_fn=<SelectBackward0>)


Thats it, now you're ready to build your own `LLoCa` networks!